**Zehra Financial Services**

_APPv_0_

This app derive returns, std, betas and correlation matrix - it uses data since the start date (or the date of the newest asset).


1. Choose the initial date for the analysis and the risk-free rate.

In [6]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact
import datetime

from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from pypfopt import risk_models, expected_returns, plotting
from pypfopt import black_litterman, risk_models
from pypfopt import EfficientFrontier, objective_functions
from pypfopt import plotting
from pypfopt import CLA

In [5]:
style = {'description_width': 'initial'}

startdate=widgets.DatePicker(
    description='Initial Date',
    value=datetime.datetime(2000, 1, 1),
    disabled=False
)

enddate=widgets.DatePicker(
    description='End Date',
    value=datetime.datetime.now(),
    disabled=False
)

rf=widgets.FloatSlider(
    value=0.03,
    min=0,
    max=0.1,
    step=0.005,
    description='Annual Risk Free Rate',style=style,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
)

tickers=widgets.Text(
    value='VFINX,VBMFX',
    placeholder='Insert tickers separated by comma', style=style,
    description='Assets (tickers):',
    disabled=False
)

NameError: name 'style' is not defined

In [ ]:
def AssetPerformance(Ticker_list,start_date,end_date,risk_free):
  #Alternative is get Yahoo Data now
  dados_yahoo = yf.download(tickers=Ticker_list, start=start_date, end=end_date)['Adj Close']
  dados_yahoo.dropna(inplace=True)
  mkt = yf.download(tickers='^GSPC', start=dados_yahoo.index[0])['Adj Close']
  mkt_return = (mkt[-1]/mkt[1])**(1/(len(mkt)-1))-1
  mkt_annual_ret = (1+mkt_return)**(251)-1
  mkt = pd.DataFrame(mkt)

  dados_yahoo.dropna(inplace=True)

  #analysis of data and parameters
  df = pd.DataFrame(dados_yahoo)
  returns = df.pct_change()
  cor = returns.corr()

  """ First analyse current portfolio """
  #Define which model to use for return andrisk
  mu = expected_returns.mean_historical_return(df, compounding=True)

  #Define which model to use to calculate the covariance matrix
  S = risk_models.sample_cov(df)

  #Portfolio assets parameters
  assets = pd.DataFrame(columns=['Returns'])
  assets['Returns'] = mu
  assets['Risk (Std)'] = np.diag(S)**0.5
  assets['Sharpe Ratio'] = (assets['Returns']-risk_free)/assets['Risk (Std)']

  #Add betas
  df1 = pd.DataFrame(df)
  df1['SP500'] = mkt
  Beta_S = risk_models.exp_cov(df1)
  cov = Beta_S
  betas = cov['SP500'] / cov.loc['SP500','SP500']
  assets['Betas*'] = betas
 
  #plot Correlation between assets
  sns.set(rc={'figure.figsize':(11.7,8.27)})
  cor_plot = sns.heatmap(cor, cmap="YlGnBu",annot=True);

  print('Analysis due to assets data availability starts at ' + str(df1.index[0]))
  print()
  return display(assets),cor_plot;

In [ ]:
interact(AssetPerformance,Ticker_list=tickers,start_date=startdate,end_date=enddate,risk_free=rf)